In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

In [7]:
df1 = pd.read_excel("MC38-d1.xlsx")
df2 = pd.read_excel("MC38-d2.xlsx")
df4 = pd.read_excel("MC38-d4.xlsx")
df5 = pd.read_excel("MC38-d5.xlsx")

df = pd.concat([df1, df2, df4, df5], ignore_index=True)

In [8]:
excel_file1 = pd.ExcelFile("MC38-d1.xlsx")
excel_file2 = pd.ExcelFile("MC38-d2.xlsx")
excel_file4 = pd.ExcelFile("MC38-d4.xlsx")
excel_file5 = pd.ExcelFile("MC38-d5.xlsx")


print("Day 1 excel file:", excel_file1.sheet_names)
print("Day 2 excel file:", excel_file2.sheet_names)
print("Day 4 excel file:", excel_file4.sheet_names)
print("Day 5 excel file:", excel_file5.sheet_names)

Day 1 excel file: ['plate1', 'plate2']
Day 2 excel file: ['plate1', 'plate2']
Day 4 excel file: ['plate1', 'plate2']
Day 5 excel file: ['plate1', 'plate2']


In [9]:
def find_all_tables_in_sheet(sheet_df, anchor="<>"):

    mask = sheet_df.astype(str).applymap(lambda x: x.strip() == anchor)
    positions = np.argwhere(mask.values)

    if len(positions) == 0:
        return []

    def is_empty(x):
        return pd.isna(x) or (isinstance(x, str) and x.strip() == "")

    def clean_header(x):
        x = str(x).strip()
        try:
            x = float(x)
            return int(x) if x.is_integer() else x
        except ValueError:
            return x

    tables = []

    for anchor_row, anchor_col in positions:

        end_col = anchor_col + 1
        while end_col < sheet_df.shape[1] and not sheet_df.iloc[anchor_row + 1:, end_col].apply(is_empty).all():
            end_col += 1

        end_row = anchor_row + 1
        while end_row < sheet_df.shape[0] and not sheet_df.iloc[end_row, anchor_col + 1:end_col].apply(is_empty).all():
            end_row += 1

        extract_header = sheet_df.iloc[anchor_row, anchor_col + 1:end_col]
        extract_index  = sheet_df.iloc[anchor_row + 1:end_row, anchor_col]

        cols = [clean_header(x) for x in extract_header]
        index = extract_index.fillna("").astype(str).str.strip().tolist()

        table = sheet_df.iloc[anchor_row + 1:end_row, anchor_col + 1:end_col].copy()
        table.columns = cols
        table.index = index

        tables.append(table)

    return tables

In [53]:
files = ["MC38-d1.xlsx", "MC38-d2.xlsx", "MC38-d4.xlsx", "MC38-d5.xlsx"]
sheet_names = ["plate1", "plate2"]

all_tables = {}

for file in files:
    sheets = pd.read_excel(file, sheet_name=sheet_names, header=None)

    for sheet in sheet_names:
        tables_in_this_sheet = find_all_tables_in_sheet(sheets[sheet], anchor="<>")

        for i, t in enumerate(tables_in_this_sheet, start=1):
            key = f"{file} | {sheet} | table{i}"
            all_tables[key] = t

print("tables found:", len(all_tables))

for name, t in all_tables.items():
    print("\n=== ", name, " ===")
    display(t)

tables found: 8

===  MC38-d1.xlsx | plate1 | table1  ===


/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/4123457341.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = sheet_df.astype(str).applymap(lambda x: x.strip() == anchor)
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/4123457341.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = sheet_df.astype(str).applymap(lambda x: x.strip() == anchor)
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/4123457341.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = sheet_df.astype(str).applymap(lambda x: x.strip() == anchor)
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/4123457341.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = sheet_df.astype(str).applymap(lambda x: x.strip() == anchor)
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934

,2,3,4,5,6,7,8,9,10,11
B,114,336.0,206.0,96,266,135.0,118.0,197,114.0,188.0
C,118,113.0,180.0,181,209,176.0,98.0,252,367.0,95.0
D,226,457.0,507.0,375,249,391.0,323.0,205,393.0,481.0
E,247,320.0,249.0,240,442,311.0,435.0,311,340.0,420.0
F,3,-4.0,7.0,0,-3,0.0,-1.0,9,2.0,1.0
G,77,91.0,95.0,2,-1,-1.0,2.0,0,-4.0,7.0



===  MC38-d1.xlsx | plate2 | table1  ===


,2,3,4,5,6,7,8,9,10,11
B,317,263.0,485.0,298,236,406.0,456.0,325,388.0,370.0
C,295,477.0,373.0,326,342,243.0,186.0,188,227.0,212.0
D,146,359.0,228.0,194,288,281.0,396.0,486,141.0,120.0
E,337,136.0,284.0,146,181,193.0,397.0,200,195.0,365.0


...

,2,3,4,5,6,7,8,9,10,11
B,190,220.0,406.0,318,373,365.0,293.0,288,435.0,528.0
C,406,479.0,239.0,328,406,282.0,96.0,120,115.0,111.0
D,187,805.0,491.0,525,466,712.0,840.0,983,186.0,208.0
E,837,195.0,555.0,339,472,381.0,867.0,583,553.0,858.0
F,3,6.0,0.0,4,10,1.0,12.0,12,17.0,8.0
G,9,4.0,9.0,5,8,0.0,8.0,5,2.0,0.0



===  MC38-d5.xlsx | plate1 | table1  ===


,2,3,4,5,6,7,8,9,10,11
B,116,413.0,269.0,145,395,225.0,108.0,269,130.0,352.0
C,203,153.0,210.0,375,366,200.0,98.0,322,356.0,85.0
D,177,657.0,936.0,745,446,458.0,714.0,423,599.0,979.0
E,626,835.0,419.0,483,1008,754.0,955.0,832,540.0,948.0
F,4,7.0,5.0,2,0,0.0,15.0,3,0.0,-2.0
G,152,137.0,134.0,-9,0,2.0,1.0,-4,5.0,-2.0



===  MC38-d5.xlsx | plate2 | table1  ===


,2,3,4,5,6,7,8,9,10,11
B,139,194.0,325.0,209,293,246.0,203.0,288,293.0,364.0
C,281,399.0,140.0,244,231,193.0,56.0,55,75.0,64.0
D,151,750.0,447.0,515,439,641.0,819.0,834,182.0,123.0
E,813,136.0,501.0,329,422,325.0,811.0,579,547.0,678.0
F,-3,0.0,0.0,-5,2,-1.0,5.0,9,-5.0,6.0
G,-5,-6.0,-5.0,-8,3,-2.0,-3.0,-5,12.0,1.0


In [54]:
# 1) Convert each table to long format: Row, Column, Value + Day + Plate
long_list = []

for key, t in all_tables.items():
    # key looks like: "MC38-d1.xlsx | plate1 | table1"
    parts = [p.strip() for p in key.split("|")]
    file_name = parts[0]      # MC38-d1.xlsx
    plate_name = parts[1]     # plate1 or plate2

    # get day from filename (MC38-d1.xlsx -> d1)
    day = file_name.split("-")[1].replace(".xlsx", "")   # "d1", "d2", "d4", "d5"

    # t is a DataFrame with:
    # index = row letters (B,C,D,E, ...)
    # columns = numbers (2..11)
    temp = t.copy()

    # make sure column labels are clean integers if possible
    temp.columns = pd.to_numeric(temp.columns, errors="ignore")

    # turn wide table into long table
    temp_long = temp.stack().reset_index()
    temp_long.columns = ["Row", "Column", "Value"]

    # add day and plate columns
    temp_long["Day"] = day
    temp_long["Plate"] = plate_name

    long_list.append(temp_long)

# combine everything
long_df = pd.concat(long_list, ignore_index=True)

# 2) Pivot so you get d1, d2, d4, d5 as separate columns (like your sample)
final_df = (
    long_df
    .pivot_table(index=["Row", "Column", "Plate"], columns="Day", values="Value", aggfunc="first")
    .reset_index()
)


# optional: sort nicely
final_df = final_df.sort_values(["Plate", "Row", "Column"]).reset_index(drop=True)

final_df


/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/2766129157.py:19: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/2766129157.py:19: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/2766129157.py:19: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/2766129157.py:19:

Day,Row,Column,Plate,d1,d2,d4,d5
0,B,2,plate1,114,169,146,116
1,B,3,plate1,336.0,577.0,441.0,413.0
2,B,4,plate1,206.0,370.0,313.0,269.0
3,B,5,plate1,96,144,176,145
4,B,6,plate1,266,475,415,395
5,B,7,plate1,135.0,259.0,286.0,225.0
6,B,8,plate1,118.0,114.0,121.0,108.0
7,B,9,plate1,197,323,274,269
8,B,10,plate1,114.0,159.0,123.0,130.0
9,B,11,plate1,188.0,413.0,381.0,352.0


In [82]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [56]:
long_list = []

for key, t in all_tables.items():
    parts = [p.strip() for p in key.split("|")]
    file_name = parts[0]      
    plate_name = parts[1]    

    day = file_name.split("-")[1].replace(".xlsx", "")

    temp = t.copy()
    temp.columns = pd.to_numeric(temp.columns, errors="ignore")

    temp_long = temp.stack().reset_index()
    temp_long.columns = ["Row", "Column", "Value"]

    temp_long["Day"] = day
    temp_long["Plate"] = plate_name

    long_list.append(temp_long)

long_df = pd.concat(long_list, ignore_index=True)
final_df = (
    long_df
    .pivot_table(
        index=["Row", "Column", "Plate"],
        columns="Day",
        values="Value",
        aggfunc="first"
    )
    .reset_index()
)

final_df = final_df[final_df["Row"].isin(["B", "C", "D", "E"])]
final_df = final_df.reset_index(drop=True)
final_df = final_df.sort_values(["Plate", "Row", "Column"]).reset_index(drop=True)
final_df["ratio d2/d1"] = final_df["d2"] / final_df["d1"]
final_df["ratio d4/d1"] = final_df["d4"] / final_df["d1"]
final_df["ratio d5/d1"] = final_df["d5"] / final_df["d1"]
final_df = final_df.rename_axis(None, axis=1)


final_df


/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/1962286729.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/1962286729.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/1962286729.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/1962286729.py:11:

,Row,Column,Plate,d1,d2,d4,d5,ratio d2/d1,ratio d4/d1,ratio d5/d1
0,B,2,plate1,114,169,146,116,1.482456,1.280702,1.017544
1,B,3,plate1,336.0,577.0,441.0,413.0,1.717262,1.3125,1.229167
2,B,4,plate1,206.0,370.0,313.0,269.0,1.796117,1.519417,1.305825
3,B,5,plate1,96,144,176,145,1.5,1.833333,1.510417
4,B,6,plate1,266,475,415,395,1.785714,1.56015,1.484962
5,B,7,plate1,135.0,259.0,286.0,225.0,1.918519,2.118519,1.666667
6,B,8,plate1,118.0,114.0,121.0,108.0,0.966102,1.025424,0.915254
7,B,9,plate1,197,323,274,269,1.639594,1.390863,1.365482
8,B,10,plate1,114.0,159.0,123.0,130.0,1.394737,1.078947,1.140351
9,B,11,plate1,188.0,413.0,381.0,352.0,2.196809,2.026596,1.87234


In [ ]:
#d1 plate 1 real time glow average data
#d2 plate 1 real time glow average data
#d4 plate 1 real time glow average data
#d5 plate 1 real time glow average data
# d1_average = (77+91+95)/3
# d2_average = (204+195+181)/3
# d4_average = (140+193+154)/3
# d5_average = (152+137+134)/3
# print (d1_average, d2_average, d4_average, d5_average)

87.66666666666667 193.33333333333334 162.33333333333334 141.0


In [89]:
long_list = []

for key, t in all_tables.items():
    parts = [p.strip() for p in key.split("|")]
    file_name = parts[0]      
    plate_name = parts[1]    

    day = file_name.split("-")[1].replace(".xlsx", "")

    temp = t.copy()
    temp.columns = pd.to_numeric(temp.columns, errors="ignore")

    temp_long = temp.stack().reset_index()
    temp_long.columns = ["Row", "Column", "Value"]

    temp_long["Day"] = day
    temp_long["Plate"] = plate_name

    long_list.append(temp_long)

long_df = pd.concat(long_list, ignore_index=True)
final_df = (
    long_df
    .pivot_table(
        index=["Row", "Column", "Plate"],
        columns="Day",
        values="Value",
        aggfunc="first"
    )
    .reset_index()
)

final_df = final_df[final_df["Row"].isin(["B", "C", "D", "E"])]
final_df = final_df.reset_index(drop=True)
final_df = final_df.sort_values(["Plate", "Row", "Column"]).reset_index(drop=True)
final_df = final_df.rename_axis(None, axis=1)

# subtract averages from d1, d2, d4, d5
# final_df["d1"] = final_df["d1"] - d1_average
# final_df["d2"] = final_df["d2"] - d2_average
# final_df["d4"] = final_df["d4"] - d4_average
# final_df["d5"] = final_df["d5"] - d5_average

# ratios
final_df["ratio d2/d1"] = final_df["d2"] / final_df["d1"]
final_df["ratio d4/d1"] = final_df["d4"] / final_df["d1"]
final_df["ratio d5/d1"] = final_df["d5"] / final_df["d1"]

# add column for conditions of well plate
def get_condition(row):
    plate = str(row["Plate"]).strip().lower()
    r = str(row["Row"]).strip().upper()

    if plate == "plate1":
        if r in ["B", "C"]:
            return "washer + collagen"
        if r in ["D", "E"]:
            return "cuboids only"

    if plate == "plate2":
        if r in ["B", "C"]:
            return "washer + cuboids"
        if r in ["D", "E"]:
            return "collagen only"

    return np.nan

final_df["Condition"] = final_df.apply(get_condition, axis=1)
final_df


final_df

/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/3250325732.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/3250325732.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/3250325732.py:11: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  temp.columns = pd.to_numeric(temp.columns, errors="ignore")
/var/folders/34/n0768sv507xdn1r6jt0yv6600000gn/T/ipykernel_42934/3250325732.py:11:

,Row,Column,Plate,d1,d2,d4,d5,ratio d2/d1,ratio d4/d1,ratio d5/d1,Condition
0,B,2,plate1,114,169,146,116,1.482456,1.280702,1.017544,washer + collagen
1,B,3,plate1,336.0,577.0,441.0,413.0,1.717262,1.3125,1.229167,washer + collagen
2,B,4,plate1,206.0,370.0,313.0,269.0,1.796117,1.519417,1.305825,washer + collagen
3,B,5,plate1,96,144,176,145,1.5,1.833333,1.510417,washer + collagen
4,B,6,plate1,266,475,415,395,1.785714,1.56015,1.484962,washer + collagen
5,B,7,plate1,135.0,259.0,286.0,225.0,1.918519,2.118519,1.666667,washer + collagen
6,B,8,plate1,118.0,114.0,121.0,108.0,0.966102,1.025424,0.915254,washer + collagen
7,B,9,plate1,197,323,274,269,1.639594,1.390863,1.365482,washer + collagen
8,B,10,plate1,114.0,159.0,123.0,130.0,1.394737,1.078947,1.140351,washer + collagen
9,B,11,plate1,188.0,413.0,381.0,352.0,2.196809,2.026596,1.87234,washer + collagen


In [72]:
final_df.to_excel("MC38_data_analysis(1).xlsx", index=False)

In [90]:
# Mask bad wells by setting their data to NaN, bad wells represent wells that have no cuboids or 2 cuboids
data_cols = [
    "d1", "d2", "d4", "d5",
    "ratio d2/d1", "ratio d4/d1", "ratio d5/d1",
    "Condition"
]
bad_wells = [
    ("plate1", "B", 3),
    # ("plate1", "C", 6),
    ("plate1", "C", 8),
    ("plate1", "C", 10),
    # ("plate1", "D", 8),
    # ("plate1", "E", 2),
    # ("plate1", "E", 7),
    # ("plate2", "C", 6),
    ("plate2", "D", 2),
    ("plate2", "D", 6),
    ("plate2", "D", 11),
    ("plate2", "E", 2),
    ("plate2", "E", 3),
    ("plate2", "E", 8),
]

for plate, row, col in bad_wells:
    mask = (
        (final_df["Plate"] == plate) &
        (final_df["Row"] == row) &
        (final_df["Column"] == col)
    )
    final_df.loc[mask, data_cols] = np.nan
final_df 


,Row,Column,Plate,d1,d2,d4,d5,ratio d2/d1,ratio d4/d1,ratio d5/d1,Condition
0,B,2,plate1,114,169,146,116,1.482456,1.280702,1.017544,washer + collagen
1,B,3,plate1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B,4,plate1,206.0,370.0,313.0,269.0,1.796117,1.519417,1.305825,washer + collagen
3,B,5,plate1,96,144,176,145,1.5,1.833333,1.510417,washer + collagen
4,B,6,plate1,266,475,415,395,1.785714,1.56015,1.484962,washer + collagen
5,B,7,plate1,135.0,259.0,286.0,225.0,1.918519,2.118519,1.666667,washer + collagen
6,B,8,plate1,118.0,114.0,121.0,108.0,0.966102,1.025424,0.915254,washer + collagen
7,B,9,plate1,197,323,274,269,1.639594,1.390863,1.365482,washer + collagen
8,B,10,plate1,114.0,159.0,123.0,130.0,1.394737,1.078947,1.140351,washer + collagen
9,B,11,plate1,188.0,413.0,381.0,352.0,2.196809,2.026596,1.87234,washer + collagen


In [75]:
final_df.to_excel("MC38_data_analysis_masked.xlsx", index=False)

In [ ]:
conditions = [
    "washer + collagen",
    "cuboids only",
    "washer + cuboids",
    "collagen only"
]

day_to_ratio = {
    "d1": "ratio d1/d1",
    "d2": "ratio d2/d1",
    "d4": "ratio d4/d1",
    "d5": "ratio d5/d1",
}

tables_by_day = {}

for day, ratio_col in day_to_ratio.items():
    tmp = final_df[["Condition", ratio_col]].copy()
    tmp = tmp.dropna(subset=["Condition", ratio_col])

    values_dict = {}
    max_len = 0

    for cond in conditions:
        vals = tmp.loc[tmp["Condition"] == cond, ratio_col].tolist()
        values_dict[cond] = vals
        max_len = max(max_len, len(vals))

    day_table = pd.DataFrame(index=range(1, max_len + 1), columns=conditions)

    for cond in conditions:
        vals = values_dict[cond]
        day_table.loc[1:len(vals), cond] = vals

    day_table.index.name = ratio_col

    tables_by_day[day] = day_table

for day in ["d1", "d2", "d4", "d5"]:
    print("\n====", day.upper(), "====")
    display(tables_by_day[day])


==== D1 ====


,washer + collagen,cuboids only,washer + cuboids,collagen only
ratio d1/d1,,,,
1,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0



==== D2 ====


,washer + collagen,cuboids only,washer + cuboids,collagen only
ratio d2/d1,,,,
1,1.482456,1.159292,0.905363,1.537604
2,1.796117,1.56674,0.965779,1.583333
3,1.5,1.56213,1.175258,1.85567
4,1.785714,1.413333,1.473154,1.907473
5,1.918519,1.522088,1.483051,1.772727
6,0.966102,1.217391,1.187192,1.790123
7,1.639594,1.399381,0.916667,1.77305
8,1.394737,1.487805,1.575385,1.588028
9,2.196809,1.386768,1.610825,1.356164



==== D4 ====


,washer + collagen,cuboids only,washer + cuboids,collagen only
ratio d4/d1,,,,
1,1.280702,0.809735,0.599369,2.24234
2,1.519417,1.796499,0.836502,2.153509
3,1.833333,1.930966,0.837113,2.706186
4,1.56015,1.8,1.067114,2.533808
5,2.118519,1.674699,1.580508,2.121212
6,1.025424,1.480818,0.899015,2.022634
7,1.390863,2.092879,0.642544,1.319149
8,1.078947,1.95122,0.886154,1.954225
9,2.026596,1.70229,1.121134,2.321918



==== D5 ====


,washer + collagen,cuboids only,washer + cuboids,collagen only
ratio d5/d1,,,,
1,1.017544,0.783186,0.438486,2.089136
2,1.305825,1.437637,0.737643,1.960526
3,1.510417,1.846154,0.670103,2.654639
4,1.484962,1.986667,0.701342,2.281139
5,1.666667,1.791165,1.241525,2.068182
6,0.915254,1.171355,0.605911,1.716049
7,1.365482,2.210526,0.445175,1.29078
8,1.140351,2.063415,0.886154,1.764085
9,1.87234,1.524173,0.755155,2.253425


In [92]:
with pd.ExcelWriter("conditions_ratios table(2).xlsx", engine="xlsxwriter") as writer:
    for day, df in tables_by_day.items():
        df.to_excel(writer, sheet_name=day)

In [98]:
days = ["d1", "d2", "d4", "d5"]

conditions = [
    "washer + collagen",
    "cuboids only",
    "washer + cuboids",
    "collagen only"
]

tables_by_condition = {}

for cond in conditions:
    tmp = final_df.loc[final_df["Condition"] == cond, days]

    values = {}
    max_len = 0

    for day in days:
        vals = tmp[day].dropna().tolist()
        values[day] = vals
        max_len = max(max_len, len(vals))

    cond_table = pd.DataFrame(
        index=days,
        columns=range(1, max_len + 1)
    )

    for day in days:
        cond_table.loc[day, 1:len(values[day])] = values[day]

    cond_table.index.name = cond
    tables_by_condition[cond] = cond_table

for cond, df in tables_by_condition.items():
    print("\n====", cond.upper(), "====")
    display(df)



==== WASHER + COLLAGEN ====


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
washer + collagen,,,,,,,,,,,,,,,,,
d1,114,206.0,96,266,135.0,118.0,197,114.0,188.0,118,113.0,180.0,181,209,176.0,252,95.0
d2,169,370.0,144,475,259.0,114.0,323,159.0,413.0,265,209.0,317.0,367,418,277.0,491,140.0
d4,146,313.0,176,415,286.0,121.0,274,123.0,381.0,237,218.0,238.0,326,382,282.0,351,161.0
d5,116,269.0,145,395,225.0,108.0,269,130.0,352.0,203,153.0,210.0,375,366,200.0,322,85.0



==== CUBOIDS ONLY ====


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
cuboids only,,,,,,,,,,,,,,,,,,,,
d1,226,457.0,507.0,375,249,391.0,323.0,205,393.0,481.0,247,320.0,249.0,240,442,311.0,435.0,311,340.0,420.0
d2,262,716.0,792.0,530,379,476.0,452.0,305,545.0,660.0,369,468.0,378.0,301,756,468.0,646.0,490,445.0,667.0
d4,183,821.0,979.0,675,417,579.0,676.0,400,669.0,978.0,572,772.0,433.0,480,1009,664.0,814.0,719,633.0,967.0
d5,177,657.0,936.0,745,446,458.0,714.0,423,599.0,979.0,626,835.0,419.0,483,1008,754.0,955.0,832,540.0,948.0



==== WASHER + CUBOIDS ====


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
washer + cuboids,,,,,,,,,,,,,,,,,,,,
d1,317,263.0,485.0,298,236,406.0,456.0,325,388.0,370.0,295,477.0,373.0,326,342,243.0,186.0,188,227.0,212.0
d2,287,254.0,570.0,439,350,482.0,418.0,512,625.0,455.0,365,537.0,415.0,480,373,400.0,209.0,145,141.0,173.0
d4,190,220.0,406.0,318,373,365.0,293.0,288,435.0,528.0,406,479.0,239.0,328,406,282.0,96.0,120,115.0,111.0
d5,139,194.0,325.0,209,293,246.0,203.0,288,293.0,364.0,281,399.0,140.0,244,231,193.0,56.0,55,75.0,64.0



==== COLLAGEN ONLY ====


,1,2,3,4,5,6,7,8,9,10,11,12,13,14
collagen only,,,,,,,,,,,,,,
d1,359.0,228.0,194,281.0,396.0,486,141.0,284.0,146,181,193.0,200,195.0,365.0
d2,552.0,361.0,360,536.0,702.0,870,250.0,451.0,198,313,298.0,421,434.0,599.0
d4,805.0,491.0,525,712.0,840.0,983,186.0,555.0,339,472,381.0,583,553.0,858.0
d5,750.0,447.0,515,641.0,819.0,834,182.0,501.0,329,422,325.0,579,547.0,678.0


In [99]:
with pd.ExcelWriter("conditions_days.xlsx", engine="xlsxwriter") as writer:
    for cond in conditions:
        df = tables_by_condition[cond]
        df.to_excel(writer, sheet_name=cond)